In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

splitter = CharacterTextSplitter(
    separator= "\n\n",
    chunk_size = 500,
    chunk_overlap = 50
)

# splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 200,
#     chunk_overlap = 50 # 문장이나 문단 분할할때 앞 조각 일부분을 가져온다.
# )

loader = UnstructuredFileLoader("./files/chapter_one.txt")

loader.load_and_split(text_splitter=splitter)

[Document(page_content='성공적인 커리어 전략: 목표 설정과 지속적인 성장의 중요성\n\n1. 서론: 성공적인 커리어란 무엇인가? 현대 사회에서 커리어는 단순한 생계를 위한 수단이 아니라, 개인의 삶을 구성하는 중요한 요소 중 하나다. 성공적인 커리어란 단순히 높은 연봉이나 직급을 의미하는 것이 아니라, 자신의 역량을 극대화하고 만족감을 느끼며 지속적으로 성장하는 과정을 포함한다. 그러나 많은 사람들이 커리어를 계획하는 과정에서 명확한 전략 없이 기회에 의존하거나 단기적인 목표에 집중하는 경향이 있다. 본 글에서는 성공적인 커리어를 구축하기 위한 핵심 전략을 목표 설정, 자기 계발, 네트워크 관리, 그리고 도전과 실패를 기회로 삼는 법을 중심으로 살펴본다.\n\n2. 본론: 성공적인 커리어 전략 1) 목표 설정과 방향성 성공적인 커리어를 위해서는 명확한 목표 설정이 필수적이다. 목표가 없는 커리어는 목적지 없이 떠도는 배와 같다. 목표를 설정할 때는 다음과 같은 SMART 원칙을 적용하는 것이 효과적이다.', metadata={'source': './files/chapter_one.txt'}),
 Document(page_content='Specific (구체적일 것): 목표는 명확해야 한다. "더 좋은 직업을 얻고 싶다"보다는 "3 년 내에 데이터 분석 전문가로 성장하고 싶다"와 같이 구체적인 목표를 설정해야 한다. Measurable (측정 가능할 것): 목표의 성취도를 평가할 수 있어야 한다. 예를 들어, "올해 안에 프로그래밍 관련 자격증 2 개를 취득한다"처럼 수치화할 수 있는 목표가 바람직하다. Achievable (달성 가능할 것): 현실적인 목표를 설정해야 한다. 너무 높은 목표는 좌절감을 줄 수 있다. Relevant (관련성이 있을 것): 목표는 자신의 장기적인 커리어 계획과 연관되어야 한다. Time-bound (기한이 있을 것): 목표를 달성할 수 있는 기한을 정해야 한다. "5 년 후에는 팀 리더가 된다"처럼 시점을 설